In [1]:
import plotly.io as pio
pio.renderers.default='notebook'

import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
from tardis.io.atom_data import AtomData
from tardis.io.atom_data.util import download_atom_data
from tardis.io.config_internal import get_data_dir
import os

ATOM_NAME = 'Helium'
ATOMIC_NUMBER = 2
ION_NUMBER = 0
NUM_SHELLS = 5 # Should be input by user

# Define energy levels
#download_atom_data('kurucz_cd23_chianti_H_He')
atom_data = AtomData.from_hdf(os.path.join(get_data_dir(), 'kurucz_cd23_chianti_H_He.h5'))
energy_levels = list(atom_data.levels.loc[ATOMIC_NUMBER, ION_NUMBER, 1:NUM_SHELLS+1].energy)

# Define dummy values of transitions and number of electrons for each
transitions = [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3), (1, 0), (2, 0), (3, 0), (2, 1), (3, 1), (3, 2)]  # transitions
transition_electrons = [5, 3, 2, 1, 1, 1, 7, 5, 1, 1, 6, 1]  # Number of electrons in each transition

# Compute transition wavelengths from difference in energy levels (converted to wavelength)
def eV_to_nm(eV):
    return 1240 * eV

transition_wavelengths = list(map(lambda jump: eV_to_nm(abs(energy_levels[jump[0]] - energy_levels[jump[1]])),
                                  transitions)) # in nm

Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

In [3]:
# Define colors for the transitions based on wavelengths
cmap = plt.get_cmap('viridis')
colors = cmap(np.linspace(0, 1, len(transition_wavelengths)))

fig = go.Figure()

# Plot energy levels
for i, energy in enumerate(energy_levels):
    fig.add_trace(go.Scatter(
        x=[0, 1],
        y=[energy, energy],
        mode='lines+text',
        text=['', f'n={i+1}'], textposition='top right',
        line=dict(color='black'),
        showlegend=False))

# Transition offset for avoiding overlap of transitions
transition_offset = 0.03  # Horizontal offset for transitions
slant = 0.05  # Horizontal offset for slanting

# Plot transitions
for i, transition in enumerate(transitions):
    start, end = transition
    x_offset = transition_electrons[i] * transition_offset  # Apply offset to x position
    slant_offset = abs(end-start) * slant  # Apply slant to x position

    fig.add_trace(go.Scatter(
    x=[0.5 + x_offset - slant_offset, 0.5 + x_offset + slant_offset],  # Apply slant offset
    y=[energy_levels[start], energy_levels[end]],  # Actual energy level values
    mode='lines',
    line=dict(color='rgb'+str(tuple((colors[i]*255).astype(int))), width=transition_electrons[i]*0.2),  # Adjust width accordingly
    name=f'{int(transition_wavelengths[i])} nm',
    hoverinfo='name',
    line_shape='linear'))

    # Add arrow
    fig.add_annotation(
        x=0.5 + x_offset - slant_offset, y=energy_levels[start],  # Start of arrow
        ax=0.5 + x_offset + slant_offset, ay=energy_levels[end],  # End of arrow
        xref='x', yref='y',
        axref='x', ayref='y',
        showarrow=True,
        arrowhead=2,  # Arrow style
        arrowsize=2,  # Make the arrowhead 2 times bigger
        arrowwidth=transition_electrons[i]*0.2,  # Adjust width accordingly
        arrowcolor='rgb'+str(tuple((colors[i]*255).astype(int))))

fig.update_layout(
    title=f"Grotrian Diagram for {ATOM_NAME}",
    title_x=0.5,
    yaxis_title="Energy Level (eV)",
    plot_bgcolor='white',
    autosize=False,
    width=700,
    height=700,
    xaxis=dict(showticklabels=False, showgrid=False,),  # Hide x-axis labels
    yaxis=dict(showgrid=False,)  # Adjust y-axis range accordingly
)


fig.show()

In [6]:
atom_data.levels.loc[ATOMIC_NUMBER, ION_NUMBER, 1:NUM_SHELLS+1].energy

atomic_number  ion_number  level_number
2              0           1               3.175454e-11
                           2               3.303013e-11
                           3               3.358819e-11
                           4               3.358820e-11
                           5               3.358840e-11
                           6               3.399504e-11
Name: energy, dtype: float64